---
# Covid-19 Analysis
---
Gruppo 3:

- Alessio Contin matr. 734792
- Stefano Santini matr. 726396

Laboratorio di Data Science

---
# Table of contents
1. [Introduzione](#introduction)
2. [Province](#province)
3. [Regioni](#regioni)

Le domande a cui vorremmo dare risposta sono le seguenti:
1. quali sono le province più colpite in base al numero di contagi?
2. quali sono le regioni più colpite? (valutazione di cosa si intende con maggiormente colpita sulla base dei vari indici epidemiologici)
3. analizzare l'andamento nel tempo delle diverse features per ogni regione
4. forecasting per valutare l'andamento delle features, ad esempio quando il numero di nuovi contagiati sarà sotto una certa soglia.


---
## Introduzione <a name="introduction"></a>

Inizialmente importiamo tutte le librerie necessarie all'esecuzione dei successivi codici.

In [ ]:
# Importazione delle librerie
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
import datetime as dt
import plotly.express as px

%matplotlib inline
%run -i "ts.py"

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits import mplot3d

from dateutil.parser import parse 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import metrics
from scipy import interpolate

#from fbprophet import Prophet
#from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as py


I dataset impiegati sono quelli di Kaggle prelevati in data 23/03/2020, relativi all'Italia, uniti ad un dataset creato, sfruttando le informazioni presenti nei seguenti siti:

Attributo relativo alla popolazione--> https://www.tuttitalia.it/regioni/popolazione/

Attributo relativo ai posti in terapia intensiva--> http://www.quotidianosanita.it/studi-e-analisi/articolo.php?articolo_id=82888

---
## Province <a name="province"></a>

Importiamo il dataset relativo alle province ed effettuiamo le prime analisi e visualizzazioni relative ad esso.

In [ ]:
#carico il dataset 
province_italy=pd.read_csv('covid19_italy_province.csv')

In [ ]:
#visualizziamo informazioni sulle colonne e sui loro tipi
province_italy.info()

In [ ]:
#visualizziamo come è formato il dataset come numero di righe e colonne
province_italy.shape

Per avere una prima impressione sul contenuto del dataset visualizziamo alcune righe.

In [ ]:
#visualizziamo le prime cinque righe
province_italy.head()

In [ ]:
#visualizziamo le ultime cinque righe
province_italy.tail()

Da una prima esplorazione del dataset, tramite le funzioni head() e tail(), notiamo che vi sono alcuni missing values, codificati come NaN nella colonna ProvinceAbbreviation, e come 'In fase di definizione/aggiornamento)' nella colonna ProvinceName.

Per avere più informazioni proviamo ad invocare la funzione describe() 

In [ ]:
province_italy.ProvinceName.describe()

Invocando la funzione describe sulla feature ProvinceName notiamo che ha 108 valori univoci, ovvero i nomi delle 107 province, più il valore che consideriamo come nullo 'In fase di definizione/aggiornamento'

Utilizziamo la funzione describe anche sull'altra feature che contiene valori nulli, ovvero 'Province Abbreviation

In [ ]:
province_italy.ProvinceAbbreviation.describe()

Notiamo che vi sono 106 valori univoci, invece di 107, numero delle province. Indaghiamo ulteriormente visualizzando la parte di dataset che ha ProvinceAbbreviation nullo.

In [ ]:
province_italy[province_italy.ProvinceAbbreviation.isnull()]

Notiamo nella riga 5 che abbiamo l'attributo ProvinceName che ha come valore Napoli, mentre la rispettiva ProvinceAbbreviation risulta nulla. Questo perchè pandas ritiene che NA non sia l'abbreviazione di Napoli ma l'indicazione di un valore nullo. 

Gestiamo il problema dei valori nulli e dell'abbreviazione di Napoli.

In [ ]:
#temp=province_italy
mask = province_italy.ProvinceName == 'Napoli'
column_name = 'ProvinceAbbreviation'
province_italy.loc[mask, column_name] = 'NAP'

Mostriamo come il dataset modificato adesso nella colonna relativa all'abbreviazione di Napoli non mostri più valore nullo ma il valore NAP

In [ ]:
province_italy[province_italy.ProvinceAbbreviation == 'NAP'].head()

In [ ]:
#Prima di eliminare i nulli dal dataframe originario, li copiamo su un
#dataset temporaneo, nel caso ci servissero per qualche analisi successiva
temp=province_italy[province_italy['ProvinceAbbreviation'].isnull()]

Abbiamo deciso di rimuovere i valori nulli, in quanto i relativi valori di TotalPositiveCases variano molto, indicando che i valori che erano in fase di aggiornamento, in parte nei giorni successivi vengono attribuiti alle rispettive province.

In [ ]:
#Rimuoviamo i valori nulli
province_italy.dropna(inplace=True)

In [ ]:
#Verifichiamo che non ci siano più valori nulli
province_italy[province_italy.ProvinceAbbreviation.isnull()].count()

Abbiamo verificato che non ci siano più valori nulli, adesso vediamo le prime informazioni sul numero totale di positivi

In [ ]:
province_italy.TotalPositiveCases.describe()

Per visualizzare meglio i dati, memorizziamo in un dataframe provvisorio i dati relativi all'ultima giornata presente, in modo da visualizzare il totale dei positivi corrente

In [ ]:
currentDf=province_italy[province_italy.Date == province_italy.Date.max()]

Ordiniamo il dataframe per il totale dei casi positivi, in ordine decrescente, in modo da selezionare la top 10 delle province per numero di contagiati attuali.

In [ ]:
#ordiniamo il dataframe
topDf=currentDf.sort_values(by='TotalPositiveCases', ascending=False)

In [ ]:
#Selezioniamo la top10 delle province
topDf=topDf.iloc[:11,:]

In [ ]:
#visualizziamo la top 10 delle province
topDf.head(10)

Per visualizzare meglio i dati, li proponiamo in formato visivo attraverso l'uso di grafici

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.barplot('ProvinceAbbreviation', 'TotalPositiveCases', data=topDf)

Dalla visualizzazione e analisi precedente notiamo che la maggior parte dei casi sono in Lombardia,Piemonte e Veneto, con alcuni anche in Marche ed Emilia Romagna. Per dare una visione globale, in rapporto anche alle province di altre regioni, diamo una visualizzazione grafica su una mappa interattiva.

In [ ]:
#Visualizziamo su una mappa, utilizzando la libreria plotly
fig = px.scatter_geo(currentDf, lat='Latitude', lon='Longitude',
                     hover_name="ProvinceAbbreviation", size="TotalPositiveCases",
                     projection="natural earth", scope='europe', color="TotalPositiveCases", center={'lat':41.902782, 'lon':12.496366})
fig.show()

Dal dataframe precedente, contenente le top 10 province all'ultima data disponibile, ricaviamo le loro abbreviazioni 

In [ ]:
provinces=[]
for i in topDf.ProvinceAbbreviation:
    provinces.append(i)
print(provinces)

Utilizzando le abbreviazioni, partendo dal dataframe originale, otteniamo un nuovo dataframe contenente le informazioni relative a ciascuna delle province presenti nella top 10 per ogni giorno presente nel dataset.

In [ ]:
#nuovo dataframe per top 10 province con arco temporale completo
topDfOverTime=province_italy[province_italy.ProvinceAbbreviation.isin(provinces)]

Mostriamo tramite grafici l'andamento del numero totale positivi nel corso dell'intero arco temporale presente per le province maggiormente colpite.

In [ ]:
#grafico andamento
fig, ax = plt.subplots(figsize=(20,10))
topDfOverTime.groupby(['Date','ProvinceAbbreviation'])['TotalPositiveCases'].max().unstack().plot(ax=ax, marker= 'o')

Visualizziamo delle boxplot.

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.boxplot('ProvinceAbbreviation', 'TotalPositiveCases', data=topDfOverTime)
plt.xlabel('ProvinceAbbreviation') # Set text for the x axis
plt.ylabel('TotalPositiveCases')# Set text for y axis

Per dare una visione globale, in rapporto anche alle province di altre regioni nell'arco temporale completo, diamo una visualizzazione grafica su una mappa interattiva.

In [ ]:
#mappa interattiva che mostra diffusione durante tutto arco temporale
fig = px.scatter_geo(province_italy, lat='Latitude', lon='Longitude', 
                     hover_name="ProvinceAbbreviation", size="TotalPositiveCases", animation_frame="Date",
                     projection="natural earth", scope='europe', color="TotalPositiveCases", center={'lat':41.902782, 'lon':12.496366})
fig.show()

---
Diamo un'occhiata all'andamento della provincia più colpita, ovvero Bergamo.

In [ ]:
#creiamo un dataframe con i dati relativi a Bergamo
BG_df=province_italy[province_italy.ProvinceAbbreviation == 'BG']

Per capire meglio l'andamento del contagio, aggiungiamo una colonna che indica il numero del giorno dall'inizio dell'epidemia.

In [ ]:
dateCount=province_italy[province_italy.ProvinceAbbreviation == 'BG'].Date.count()
ticks=range(0,dateCount,1)
BG_df=BG_df.sort_values(by='Date', ascending=True)
BG_df['DayNumber']=ticks
BG_df.head()

Visualizziamo l'andamento tramite grafici.

In [ ]:
#province_italy[province_italy.ProvinceAbbreviation == 'BG']
fig, ax = plt.subplots(figsize=(20,10))
g=sns.barplot('DayNumber', 'TotalPositiveCases', data=BG_df)

In [ ]:
#sns.lineplot(data=prova, palette="tab10", linewidth=2.5)
fig, ax = plt.subplots(figsize=(20,10))
sns.lineplot('DayNumber', 'TotalPositiveCases', data=BG_df, marker='o')

Notiamo che l'andamento segue una curva molto accentuata, quasi esponenziale, con un numero di contagi superiore anche a città più grandi come ad esempio Milano. Resta da capire perchè il numero di contagi si è concentrato in Lombardia, se c'è stato qualche evento particolare, o se c'è qualche relazione anche in base a parametri e risultati che vedremo nella sezione sulle regioni, analizzando il relativo dataset, che contiene dati aggiuntivi come il numero di decessi, di ospedalizzati, di positivi e nuovi positivi, nonchè il numero di tamponi e di posti letto in terapia intensiva. 

---
## Regioni <a name="regioni"></a>

Come nella sezione precedente, importiamo i dataset effettuiamo le prime analisi e visualizzazioni.

In [ ]:
#caricamento dataset
regione_orig = pd.read_csv('covid19_italy_region.csv',parse_dates=['Date'])
popolazioni_orig=pd.read_csv('popolazioni.csv')

In [ ]:
regione = regione_orig.copy() 
popolazioni=popolazioni_orig.copy()

Dopo aver copiato i dataset di interesse, si esegue un 'merge' in base al nome della regione.
Successivamente si visualizzano le dimensioni e le informazioni sul datset creato.

In [ ]:
regione2 = pd.merge(regione, popolazioni, left_on="RegionName", right_on="Regione").drop('Regione', axis=1)

In [ ]:
regione2.shape

In [ ]:
regione2.info()

In [ ]:
#visualizziamo primi cinque elementi
regione2.head()

In [ ]:
#visualizziamo ultimi cinque elementi
regione2.tail()

Sono stati creati nuovi attributi per meglio analizzare la situazione delle diverse regioni sotto più punti di vista.
Di seguito viene proposta una spiegazione di ogni nuova feature.

------------------------------------------------------------------------------------------------------
LEGENDA

Se a t(2) ho CurrentPositive=100 questo è uguale a NewPositive(t(0))+NewPositive(t(1))

"TotalHospitalizedPatient" + "HomeConfinement" = "CurrentPositive"

"IntensiveCare" + "HospitalizedPatient" = "TotalHospitalizedPatients"

"CurrentPositive" + "Deaths" + "Recovered" = "TotalPositiveCases"

Viene evidenziata una dipendenza tra i vari attributi.


SPIEGAZIONE VARIABILI

ratio_PIR_Pop: corrisponde al rapporto tra i posti letto in terapia intensiva a regime con tutta la popolazione (entrambi i parametri sono relativi alla regione).

ratio_IC_HP: corrisponde al rapporto tra i pazienti in terapia intensiva e quelli ospedalizzati (entrambi i parametri sono relativi alla regione).

ratio_TPC_TeP: rappresenta il rapporto tra i casi positivi totali e il numero di test (tamponi) eseguiti sulla popolazione (entrambi i parametri sono relativi alla regione).

ratio_TPC_Pop: definito anche come prevalenza, rappresenta il numero di casi positivi totali rispetto alla popolazione (entrambi i parametri sono relativi alla regione).

ratio_D_Pop: definito anche come tasso di mortalità, corrisponde ai decessi sul totale della popolazione (entrambi i parametri sono relativi alla regione).

ratio_TCP_Pop_gg: definito anche come tasso di incidenza grezzo, definisce il numero di casi in un dato intervallo temporale (entrambi i parametri sono relativi alla regione).

proportion_IC_HP_TPC: proporzione tra tutti i pazienti in ospedale e il numero di casi positivi totali (entrambi i parametri sono relativi alla regione).

proportion_HC_TPC: proporzione tra il numero di paziento ospedalizzati e il totale delle persone che hanno la patologia (entrambi i parametri sono relativi alla regione).

proportion_CPC_TPC: proporzione tra totale dei pazienti in ospedale sommato a quelli confinati a casa e il numero di casi positivi totali (entrambi i parametri sono relativi alla regione).

proportion_R_TPC: proporzione tra il numero di guariti e il numero di casi positivi totali (entrambi i parametri sono relativi alla regione).

proportion_IC_PIR: definita anche come indice di occupazione, definisce il numero di posti letto occupati, in questo caso in terapia intensiva (entrambi i parametri sono relativi alla regione).

proportion_D_TPC: definita anche come letalità, definisce il numero di decessi sul totale della popolazione (entrambi i parametri sono relativi alla regione). 

---------------------------------------------------------------------------------------------------------------

Per la creazione del tasso di incidenza è stato implementato il seguente codice in modo da avere per ogni regione e per ogni numero di giorni trascorso la rispettiva incidenza che rappresenta la frequenza di contagi.
Es: il valore ottenuto dal rapporto verrà diviso per il numero di giorni trascorsi: al primo giorno avrò una divisione per 1, al secondo per 2, al 28 per 28.

In [ ]:
#Tasso di Incidenza x 100000 abitanti
regione2.insert(regione2.shape[-1], 'ratio_TCP_Pop_gg', (regione2['TotalPositiveCases']/regione2['Popolazione'])*100000)

In [ ]:
regione2

In [ ]:
# Valutazione dell'incidenza relativa ad ogni regione per oggni giorno
a=0
b=int(regione2.shape[0]/21)
i=1

while b<regione2.shape[0]+1:  
    i=1
    for k in range(a,b):
        rowIndex = regione2.index[k]
        regione2.loc[rowIndex, 'ratio_TCP_Pop_gg'] = (regione2.loc[rowIndex, 'ratio_TCP_Pop_gg'])/i
        i=i+1
        
    a=a+int(regione2.shape[0]/21)
    b=b+int(regione2.shape[0]/21)
    

In [ ]:
regione2.head(84)

In [ ]:
regione2.tail()

Gli indici creati vengono moltiplicati per i fattori 100, 10000 a seconda dei casi, come da prassi epidemiologica, al fine di ottenere valori in percentuale o rapportati ad un dato quantitativo di abitanti.

In [ ]:
#Inserisco nella tabella creata alcuni nuovi attributi di interesse e alcune misure di incidenza epidemiologica
   
#Valore di posti Intensivi x 10000
regione2.insert(regione2.shape[-1], 'ratio_PIR_Pop', (regione2['PostiIntensivaRegime']/regione2['Popolazione'])*10000)
#Valori percentuali
regione2.insert(regione2.shape[-1],'ratio_IC_HP', (regione2['IntensiveCarePatients']/regione2['HospitalizedPatients'])*100)
regione2.insert(regione2.shape[-1],'ratio_TPC_TeP', (regione2['TotalPositiveCases']/regione2['TestsPerformed'])*100)
#Prevalenza x 10000 abitanti
regione2.insert(regione2.shape[-1], 'ratio_TPC_Pop', (regione2['TotalPositiveCases']/regione2['Popolazione'])*10000)
#Mortalità x 10000 abitanti
regione2.insert(regione2.shape[-1], 'ratio_D_Pop', (regione2['Deaths']/regione2['Popolazione'])*10000)


In [ ]:
#Valori percentuali
regione2.insert(regione2.shape[-1],'proportion_IC_HP_TPC', ((regione2['HospitalizedPatients']+regione2['IntensiveCarePatients'])/regione2['TotalPositiveCases'])*100)
regione2.insert(regione2.shape[-1],'proportion_HC_TPC', (regione2['HomeConfinement']/regione2['TotalPositiveCases'])*100)
regione2.insert(regione2.shape[-1],'proportion_CPC_TPC', (regione2['CurrentPositiveCases']/regione2['TotalPositiveCases'])*100)
regione2.insert(regione2.shape[-1],'proportion_R_TPC', (regione2['Recovered']/regione2['TotalPositiveCases'])*100)
# %Occupazione
regione2.insert(regione2.shape[-1],'proportion_IC_PIR', (regione2['IntensiveCarePatients']/regione2['PostiIntensivaRegime'])*100)
# %Letalità 
regione2.insert(regione2.shape[-1],'proportion_D_TPC', (regione2['Deaths']/regione2['TotalPositiveCases'])*100)




In [ ]:
#Visulaizziamo Le informazioni relative al dataset creato, per valutare se vi sono valori NaN
regione2.info()

Si osserva la presenza di valori nulli; questi verranno considerati in seguito. 

Per ora si considerano i dati relativi all'ultima data nota.

In [ ]:
#Seleziono gli elementi degli attributi relativi all'ultimo giorno, dato che ogni elemento è cumulativo 
regione_new = regione2[regione2["Date"] == max(regione2["Date"])].reset_index()

In [ ]:
max(regione2["Date"])
regione_new

Si prosegue con la creazione di un dataframe con i soli attributi di interesse relativi all'ultima data e per ogni regione.

In [ ]:
#Raggruppo per regione e seleziono gli attributi di interesse
regione_new2=regione_new.groupby('RegionName')['HospitalizedPatients','CurrentPositiveCases', 'Deaths', 'IntensiveCarePatients', 'Recovered', 
                                               'HomeConfinement', 'TotalPositiveCases', 'TestsPerformed', 
                                               'Popolazione', 'PostiIntensivaRegime','ratio_PIR_Pop', 'ratio_IC_HP',
                                               'ratio_TPC_TeP','ratio_TPC_Pop','ratio_D_Pop','ratio_TCP_Pop_gg',
                                               'proportion_IC_HP_TPC','proportion_HC_TPC','proportion_CPC_TPC',
                                               'proportion_R_TPC','proportion_IC_PIR','proportion_D_TPC'].max()


In [ ]:
#Stampo a video il dataframe
regione_new2

In [ ]:
#Stampo a video i dati di ogni regione, sfruttando una funzione di Pandas per evidenziare i dati sulla base della 
#loro rilevanza all'interno della colonna
regione_new2.style.background_gradient()

La tabella sopra proposta rappresenta un utile strumento per valutare l'incidenza della malattia da diversi punti di vista.
Come si osserva la Lombardia è la regione che possiede i valori più alti per quasi tutti gli indici.
Considerando gli indici ordinari, cioè quelli che non sono stati derivati (quelli epidemiologici), si osserva che le regioni più interessate sono Lombardia, Emilia Romagna, Veneto e Piemonte.
Tuttavia questi indici tengono in considerazione un solo aspetto e sono piuttosto rigidi.
Andiamo ad osservare cosa emerge dagli indici derivati:

ratio_PIR_Pop --> Emilia Romagna e Valle d'Aosta primeggiano per il numero di posti letto in terapia intensiva per 10000, decisamente inferiore rispetto a quella della Lombardia, nonostante abbiano una popolazione inferiore a quest'ultima. La Lombardia si attesta ad un livello simile a quello delle regioni centro-meridionali. 

ratio_IC_HP	--> in questo caso le regioni che hanno un percentuale maggiore di pazienti in terapia intensiva, rispetto a quelli in ospedale, sono la Basilicata e la Campania. Quindi queste due regioni hanno più pazienti in terapia intensiva di quelli in ospedale nonostante la popolazione ed il numero di contagiati siano inferiori rispetto alla Lombardia, Veneto, Piemonte.

ratio_TPC_TeP --> Lombardia, Marche e Valle d'Aosta emergono per aver il maggior numero di contagi rispetto al numero di tamponi effettuati. Tuttavia non è del tutto vero che all'aumentare del numero di tamponi effettuati in una regione, il numero di contagiati aumenta, in quanto il Veneto è al secondo posto come numero di tamponi eseguiti, ma di questi poco meno del 9% sono positivi.

ratio_TPC_Pop --> Valle d'Aosta, Lombardia, Marche e P.A. Trento hanno i valori più alti di contagi rispetto alla popolazione.
Questo porterebbe a pensare che minore è la popolazione maggiore sia il numero di contagi, quindi la rapidità risulta maggiore; tuttavia maggiore è la popolazione, maggiore è il numero di soggetti suscettibili al contagio. Per valutare questo aspetto è necessario osservare il Tasso di Incidenza.

ratio_D_Pop	--> La Lombardia ha il valore di morti per Covid x 10000 abitanti più alto. Ovviamente per quanto detto sopra maggiore è il bacino di soggetti influenzabili, maggiore è la probabilità di avere un numero alto di contagiati e quindi un alto numero di morti.

ratio_TCP_Pop_gg --> il Tasso di Incidenza evidenzia che Valle d'Aosta e Lombardia sono le più interessate e quindi le più colpite dal punto di vista della frequenza dei contagi. Questo indice mette in evidenza che, sulla base di quanto accennato sopra, sia la rapidità che il bacino di influenza sono aspetti egualmente importanti nella diffusione di una malattia.
Utile per realizzare strategie di controllo e risposta per la malattia.

proportion_IC_HP_TPC --> Piemonte, Lazio, Liguria, Molise e Abruzzo sono le più interessate dal punto di vista dei pazienti che richiedono trattamenti ospedalieri per la malattia, rispetto al numero di contagiati.
Questo fa pensare che qualora il numero di contagiati non sia eccessivo alcune regioni applichino di più la politica di cura ospedaliera, salvo il fatto dei trattamenti di terapia intensiva che di per sè sono necessari, compatibilmente alla capacità del loro sistema sanitario.

proportion_HC_TPC --> Valle d'Aosta, Sardegna e Basilicata primeggiano per il numero di paziento positivi confinati a casa, ma si osserva che la maggior parte delle regioni ha valori elevati per questo indice. Comparando i valori con quelli dell'indice precedente emerge il fatto che le regioni con valori inferiori, relativi all'indice in esame, sono quelle che hanno un maggior numero di pazienti che necessitano di cure ospedaliere.

proportion_CPC_TPC --> La Lombardia ha la percentuale minore di pazienti confinati a casa e ospedalizzati rispetto alla totalità dei casi positivi. Ciò significa che la restante percentuale, più alta rispetto alle altre regioni è composta da soggetti guariti e deceduti.

proportion_R_TPC --> La componente percentuale complementare a quella del punto precedente risulta, nel caso della Lombardia, composta per lo più dai soggetti guariti.
Nei casi più evidenti quali Emilia Romagna e Molise, si osserva che tale componente percentuale è per la maggior parte composta da soggetti deceduti.

proportion_IC_PIR --> Lombardia, Marche e Valle d'Aosta hanno la più alta saturazione dei posti di terapia intensiva rispetto al numero di contagiati. Questo parametro è di sicuro molto importante per valutare l'impatto della malattia sulle disponibilità delle risorse intensive di ogni regione. Quindi non solo le regioni più popolose hanno criticità nella terapia intensiva, ma anche quelle con un numero di abitanti inferiori. Potrebbe essere uno spunto per valutare politiche di riadeguamento regionale delle risorse in caso di situazioni critiche come quella attuale.

proportion_D_TPC --> La Lombardia, Emilia Romagna e Liguria hanno alti valori, tuttavia vale quanto riportato in 'proportion_R_TPC'

Sulla base di quanto osservato emerge che i parametri di maggior valore sono proportion_IC_PIR e ratio_TCP_Pop_gg, pertanto le regioni maggiormente interessate dalla malattia sono Lombardia e Valle d'Aosta. Tuttavia gli altri indici restano validi per comprendere le varie sfaccettature del problema.

Creazione di un nuovo dataframe per osservare i dati globali dell'Italia, relativamente alle features ordinarie.

In [ ]:
#Raggruppo per regione e seleziono gli attributi del dataset originale per ottenere una valutazione sull'italia
regione_new3=regione_new.groupby('Date')['HospitalizedPatients','CurrentPositiveCases', 'Deaths', 'IntensiveCarePatients', 'Recovered', 
                                         'HomeConfinement', 'TotalPositiveCases', 'TestsPerformed',
                                         'Popolazione', 'PostiIntensivaRegime'].sum()
           

In [ ]:
#Stampo i dati totali di tutte le regioni per avere una visione globale
regione_new3.style.background_gradient()

In [ ]:
#Realizzazione di un grafico delle dispersioni per ogni attributo
scatterplot_matrix  = pd.plotting.scatter_matrix(regione_new2, alpha=0.2, figsize=(15, 15), diagonal='kde', s=200)
for ax in scatterplot_matrix.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha('right')
plt.show()

Lo scatterplot non permette un immediata valutazione delle relazioni tra le features derivate.
Si intuisce un rapporto di dipendenza per quanto riguarda quelle ordinarie.

Si esegue un grafico heatmap per valutare meglio se è presente una correlazione.

In [ ]:
#Dato che il grafico delle dispersioni è poco utile a intuire eventuali relazioni si effettua un matrice di correlazioni
sns.set(font_scale=1.4)
plt.figure(figsize=(20,15))
sns.heatmap(regione_new2.corr(),annot=True)
plt.axes().set_title('Heatmap n°1',fontsize =20)
plt.show()

Dalla matrice di correlazione si evidenzia il fatto che è presente un'effettiva dipendenza tra gli attributi appartenenti al dataset originale, uniti anche ai due attributi definiti come 'Popolazione' e 'PostiIntensivaRegime'. In questo contesto la correlazione è diretta. 
Per le misure di incidenza si evidenziano casi in cui si ha una correlazione diretta, ma anche casi in cui tale correlazione è indiretta, ciò è dovuto alla natura dell'operazione matematica di rapporto e proporzione. A tal proposito si nota la presenza anche di bassi valori di correlazione compresi tra circa -0.4 e 0.4.

Si procede alla creazione di variabili per poter eseguire una visualizzazione grafica di alcune features del dataset e per meglio comprendere gli andamenti di queste, come riportato nella tabella 'background'

In [ ]:
#Esecuzione di un sort dei dati all'interno di ogni attributo
pop= popolazioni.sort_values(by= ['Popolazione'], ascending=False)
positive_cases = regione_new.sort_values(by = ['CurrentPositiveCases'], ascending=False)
intensive_cases= regione_new.sort_values(by = ['IntensiveCarePatients'], ascending=False)
recovered= regione_new.sort_values(by = ['Recovered'], ascending=False)
home_confinement = regione_new.sort_values(by = ['HomeConfinement'], ascending=False)
deaths= regione_new.sort_values(by = ['Deaths'], ascending=False)

In [ ]:
#settaggio paerametri per il plot
sns.set(style = "darkgrid")

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(pop.Regione, pop.Popolazione, color='b')
plt.show()

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(positive_cases.RegionName, positive_cases.CurrentPositiveCases, color='b')
plt.show()

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(intensive_cases.RegionName, intensive_cases.IntensiveCarePatients, color='b')
plt.show()

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(recovered.RegionName, recovered.Recovered, color='b')
plt.show()

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(home_confinement.RegionName, home_confinement.HomeConfinement, color='b')
plt.show()

plt.figure(figsize=(10,5))
plt.xticks(rotation = 90)
sns.barplot(deaths.RegionName, deaths.Deaths, color='b')
plt.show()

I grafici eseguiti sulle variabili ordinarie confermano quanto osservato nella tabella precedente di 'background'

A supporto delle osservazioni proposte per le features derivate, si esegue la rappresentazione grafica dei valori di queste.

In [ ]:
# Risultati in %
regione_new2['proportion_IC_HP_TPC'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati in %
regione_new2['proportion_HC_TPC'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati in %
regione_new2['proportion_CPC_TPC'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati in %
regione_new2['proportion_R_TPC'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati di Letalità in %
regione_new2['proportion_D_TPC'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati in %
regione_new2['ratio_IC_HP'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati in %
regione_new2['ratio_TPC_TeP'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati di Prevalenza x 10000 abitanti
regione_new2['ratio_TPC_Pop'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati dei Posti di Terapia Intensiva x 10000
regione_new2['ratio_PIR_Pop'].sort_values().plot.barh(figsize=(10,10))

In [ ]:
# Risultati di Occupazione in %
regione_new2['proportion_IC_PIR'].sort_values().plot.barh(figsize=(10,10))

Le raffigurazioni sugli indici evidenziano sì, la prevalenza della Lombardia nelle prime posizioni, ma mostrano come a seconda dell'indice considerato possono variare le regioni nelle successive posizioni. 
Sembra che le regioni del Nord siano le più interessate, concordemente al fatto che l'epidemia si è sviluppata proprio in quelle zone. Tuttavia appare non scontato il fatto che la regione con più contagi sia quella effettivamente più colpita; si veda infatti il numero di posti in terapia intensiva rispetto ai pazienti effettivamente bisognosi di queste cure: la Valle d'Aosta e le Marche risultano analogamente interessata come la Lombardia, nonostante il loro numero di contagi sia decisamente inferiore. 

Di seguito vengono valutati gli andamenti di alcune features per regione. Si rimanda alla sezione relativa alle Time Series per approfondimenti.

In [ ]:
#Valutazione dell'anadamento dei pazienti ospedalizzati nell'arco di tempo presente nel dataset
fig, ax = plt.subplots(figsize=(20,10))
regione.groupby(['Date','RegionName'])['HospitalizedPatients'].max().unstack().plot(ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
regione.groupby(['Date','RegionName'])['IntensiveCarePatients'].max().unstack().plot(ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
regione.groupby(['Date','RegionName'])['TestsPerformed'].max().unstack().plot(ax=ax)

Il comportamento nel tempo degli attributi considerati evidenzia, in due casi, un andamento simil-esponenziale per le regioni, in modo particolare per la Lombardia e il Veneto(grafico n°3).
In un caso (grafico n°2) l'andamento sembra più proporzionale.
In tutte le raffigurazioni grafiche la Lombardia si presenta come la regione più interessata. 

Ora verranno considerati i valori NaN trovati all'inizio della trattazione.

In [ ]:
regione2

In [ ]:
pd.isnull(regione2)

In [ ]:
# I dati Nan sono ottenuti da una divisione per il valore zero. La sostituzione porta all'inserimento del valore 0 
regione3=regione2.fillna(0)
#datset regione3 è quello completo
regione3

In [ ]:
regione3.info()

Andiamo a valutare se sono presenti valori 'inf' e dove sono; qualora ci fossero questi verranno sostituiti con 0.


In [ ]:
df_attr1=regione3.iloc[:, 7:]
infi=np.isinf(df_attr1)
np.where(np.isinf(df_attr1))
df_attr1.index[np.isinf(df_attr1).any(1)]

In [ ]:
df_attr1.columns.to_series()[np.isinf(df_attr1).any()]

In [ ]:
print(regione3.iloc[47,21])
print(regione3.iloc[48,21])
print(regione3.iloc[367,22])

Dato l'esiguo numero di valori infiniti si procede con una sostituzione manuale.
Si procede poi con un controllo sull'effettiva sostituzione.

In [ ]:
regione3.iloc[47,21] = 0.0
regione3.iloc[48,21] = 0.0
regione3.iloc[367,22] = 0.0

In [ ]:
df_attr1=regione3.iloc[:, 7:]
infi=np.isinf(df_attr1)
np.where(np.isinf(df_attr1))
df_attr1.index[np.isinf(df_attr1).any(1)]

Si valuta se la correlazione tra le fatures, valutata nell'heatmap precedente solo sull'ultimo record cumulativo, è confermata per tutti i record del dataset.

In [ ]:
sns.set(font_scale=1.4)
plt.figure(figsize=(20,15))
sns.heatmap(regione3.iloc[:,7:].corr(),annot=True)
plt.axes().set_title('Heatmap n°2',fontsize =20)
plt.show()

Appare evidente la correlazione tra le features ordinarie e si evidenziano alcune correlazioni significative anche tra le features derivate.
Es: 'proportion_IC_PIR' e 'ratio_TCP_Pop_gg' = 86%

Osservando la sotto-mappa relativa alle features derivate si osserva che la correlazione più alta è proprio tra i due attributi citati nell'esempio qui sopra. Si tralasciano nella sotto-mappa in esame gli attributi con il 98%, in quanto strettamente dipendenti per la relazione matematica.
Da un'analisi retrospettiva emerge quindi che due indici importanti sono appunto 'proportion_IC_PIR' e 'ratio_TCP_Pop_gg', che possono essere impiegati per valutare le regioni più colpite dalla malattia. Appare quindi confermato quanto esposto in precedenza nella tabella di background.

Si prosegue ora con l'esecuzione dell'analisi PCA, prima sulle features ordinarie e poi su quelle derivate.
Lo split dell'analisi in due parti serve per evitare una eccessiva richiesta di componenti principali per coprire la varianza totale.
La PCA con features ordinarie viene eseguita senza gli attributi 'Popolazione' e 'PostiIntensivaRegfime', perchè sono elementi costanti per ogni regione e andrebbero a fornire un identificativo uguale, per finalità, alla label.

In [ ]:
df_attr=regione3.iloc[:, 7:17]

In [ ]:
df_attr

Viene creata una variabile relativa a quelle che sono definibili delle label per ogni regione, cioè i codici regionali.

In [ ]:
df_reg=regione3.iloc[:, 3:4]

I dati vengono scalati con un MaxMin scaler, dato che dalla letteratura si evidenzia una robustezza dell'algoritmo nello scaling in un dato range.

In [ ]:
scaler = MinMaxScaler(feature_range=[0, 1])
data_rescaled = scaler.fit_transform(df_attr)

Si esegue la PCA al fine di valutare il numero di componenti necessarie per coprire la Varianza totale

In [ ]:
#Fitting dell'algoritmo di PCA con il dataset
pca = PCA().fit(data_rescaled)
#Plotting della somma cumulativa della Explained Variance
plt.figure(figsize=(10,10))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Dataset Explained Variance')
plt.show()

Dal grafico emerge che considerando 5 componenti si riesce a coprire tutta la varianza del dataset (circa 100%).
Al fine di eseguire anche un'analisi grafica si considerano 3 componenti principali.
Vengono eseguiti quindi i grafici 2D e 3D per la PCA, coprendo una Varianza totale > 98%

In [ ]:
pca = PCA(n_components=3, svd_solver='full')
principalComponents = pca.fit_transform(data_rescaled)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2',
                                                                  'principal component 3'])

Vengono concatenate le label alle componenti principali.

In [ ]:
finalDf = pd.concat([principalDf, df_reg], axis = 1)

In [ ]:
finalDf

In [ ]:
#2D PCA visulaization
fig = plt.figure(figsize = (20,20))
ax = fig.add_subplot(1,1,1)
#ax = plt.axes(projection='3d') 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
#ax.set_zlabel('Principal Component 3', fontsize = 20)
ax.set_title('2 component PCA', fontsize = 25)
targets = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
colors = ['red', 'green', 'blue', 'yellow','black', 'c', 'crimson','chocolate', 'maroon', 'purple', 'fuchsia', 'lime', 'olive', 
          'navy', 'teal', 'aqua','burlywood','chartreuse','orange', 'pink']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['RegionCode'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 20)
ax.legend(targets)
ax.grid()


In [ ]:
#3D PCA visualization
fig = plt.figure(figsize = (20,20))
ax = plt.axes(projection='3d') 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
ax.set_zlabel('Principal Component 3', fontsize = 20)
ax.set_title('3 component PCA', fontsize = 25)
targets = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
colors = ['red', 'green', 'blue', 'yellow','black', 'c', 'crimson','chocolate', 'maroon', 'purple', 'fuchsia', 'lime', 'olive', 
          'navy', 'teal', 'aqua','burlywood','chartreuse','orange', 'pink']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['RegionCode'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , finalDf.loc[indicesToKeep, 'principal component 3']
               , c = color
               , s = 30)
ax.legend(targets)
ax.grid()


Dai grafici sembra che l'utilizzo di 2 e 3 componenti riesca a descrivere le classi relative alle regioni e a separarle adeguatamente.

Viene creato un dataframe contenente le componenti e le features esaminate.

In [ ]:
completeDf = pd.concat([principalDf, df_attr], axis = 1)

In [ ]:
completeDf = pd.concat([completeDf, df_reg], axis = 1)

In [ ]:
completeDf

Creazione del dataframe con le correlazioni tra fetaures e componenti principali e successiva rappresentazione grafica.

In [ ]:
corr_matrix = completeDf.corr()
corr_matrix

In [ ]:
fig = plt.figure(figsize=(15,15))

plt.imshow(completeDf.iloc[:, :-1].corr(), cmap = plt.cm.YlOrRd, interpolation='nearest')
plt.colorbar()
tick_marks = [i for i in range(len(completeDf.iloc[:, :-1].columns))]
plt.xticks(tick_marks, completeDf.iloc[:, :-1].columns, rotation='vertical')
plt.yticks(tick_marks, completeDf.iloc[:, :-1].columns)

In questo caso le features più correlate sono (escludendo le 'components', osservando l'asse orizzontale):
    
PC1 --> features più correlate 1, 2, 3, 5 e 9

PC2 --> features più correlate 7, 8 e 10

PC3 --> features più correlate 6, 7 e 8

Si confermano le correlazioni delle features evidenziate nell'heatmap n°1

Con il successivo comando visualizziamo in dettaglio le componenti della pca per meglio capire quali features sono rilevanti all'interno di ogni componente.

In [ ]:
print(abs( pca.components_ ))

pca.components_ ha la forma [n_components, n_features].

PC1 --> features più importanti 2,3,5 e 10

PC2 --> features più importanti 7 e 10

PC3 --> features più importanti 6,7 e 10

I passaggi eseguiti in precedenza per la PCA vengono riproposti, analizzando le sole features derivate.

In [ ]:
df_attr1=regione3.iloc[:, 19:]

In [ ]:
scaler1 = MinMaxScaler(feature_range=[0, 1])
data_rescaled1 = scaler1.fit_transform(df_attr1)

In [ ]:
#Fitting dell'algoritmo di PCA con il dataset
pca1 = PCA().fit(data_rescaled1)
#Plotting della somma cumulativa della Explained Variance
plt.figure(figsize=(10,10))
plt.plot(np.cumsum(pca1.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Dataset Explained Variance')
plt.show()

In [ ]:
pca1 = PCA(n_components=6)
principalComponents1 = pca1.fit_transform(data_rescaled1)
principalDf1 = pd.DataFrame(data = principalComponents1, columns = ['principal component 1', 'principal component 2', 
                                                                    'principal component 3', 'principal component 4', 
                                                                    'principal component 5','principal component 6'])

In [ ]:
finalDf1 = pd.concat([principalDf1, df_reg], axis = 1)

In [ ]:
#3D PCA visualization
fig = plt.figure(figsize = (20,20))
ax = plt.axes(projection='3d') 
ax.set_xlabel('Principal Component 1', fontsize = 20)
ax.set_ylabel('Principal Component 2', fontsize = 20)
ax.set_zlabel('Principal Component 3', fontsize = 20)
ax.set_title('3 component PCA', fontsize = 25)
targets = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
colors = ['red', 'green', 'blue', 'yellow','black', 'c', 'crimson','chocolate', 'maroon', 'purple', 'fuchsia', 'lime', 'olive', 
          'navy', 'teal', 'aqua','burlywood','chartreuse','orange', 'pink']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf1['RegionCode'] == target
    ax.scatter(finalDf1.loc[indicesToKeep, 'principal component 1']
               , finalDf1.loc[indicesToKeep, 'principal component 2']
               , finalDf1.loc[indicesToKeep, 'principal component 3']
               , c = color
               , s = 30)
ax.legend(targets)
ax.grid()

In [ ]:
completeDf1 = pd.concat([principalDf1, df_attr1], axis = 1)
completeDf1 = pd.concat([completeDf1, df_reg], axis = 1)

In [ ]:
corr_matrix1 = completeDf1.corr()
corr_matrix1

In [ ]:
fig = plt.figure(figsize=(15,15))

plt.imshow(completeDf1.iloc[:, :-1].corr(), cmap = plt.cm.YlOrRd, interpolation='nearest')
plt.colorbar()
tick_marks = [i for i in range(len(completeDf1.iloc[:, :-1].columns))]
plt.xticks(tick_marks, completeDf1.iloc[:, :-1].columns, rotation='vertical')
plt.yticks(tick_marks, completeDf1.iloc[:, :-1].columns)

L'analisi PCA sulle features derivate mette in evidenza il fatto che le features ordinarie sembrano più adatte ad ottenere PC meglio correlate. 
Si evidenziano conferme alle correlazioni tra le varie features derivate, riscontrate nell'heatmap n°2.

Si traccia l'andamento dei Test per l'Italia.

In [ ]:

data=regione2.groupby("Date")[['TotalPositiveCases', 'Deaths', 'Recovered','TestsPerformed','HospitalizedPatients',
                          'TotalHospitalizedPatients']].sum().reset_index()

In [ ]:
data

In [ ]:
def plot_df(data, x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.xticks(rotation = 90)
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

plot_df(data, x=data.Date, y=data.TestsPerformed, title='')

I dati sono stati raggruppati per regione; la funzione ordina per data e sposta i dati in ciascun gruppo. Infine, si applica una concatenazione per ottenere il dataframe nel suo formato originale.

In [ ]:
reg_group=regione3.groupby(["RegionName"])

In [ ]:
def lag_by_group(key, value_df):
    df = value_df.assign(group = key) 
    return (df.sort_values(by=["Date"], ascending=True)
        .set_index(["Date"])
        .shift(1)
               )

In [ ]:
dflist = [lag_by_group(g, reg_group.get_group(g)) for g in reg_group.groups.keys()]
pd.concat(dflist, axis=0).reset_index()

I grafici seguenti sono relativi ad alcune features ordinarie, prima per l'Italia, poi per ogni regione, considerando 4 lag.


In [ ]:
reg_list=regione3['RegionName'].unique()
#print(reg_list[0])

In [ ]:
from pandas.plotting import lag_plot
plt.rcParams.update({'ytick.left' : False, 'axes.titlepad':10})


#names = ['HospitalizedPatients', 'Deaths', 'IntensiveCarePatients', 'Recovered', 'HomeConfinement','TotalPositiveCases',
      #   'TestsPerformed', 'ratio_PIR_Pop', 'ratio_IC_HP', 'ratio_TPC_TeP', 'ratio_TCP_Pop_gg', 'proportion_HC_TPC',
       #  'proportion_CPC_TPC','proportion_R_TPC', 'proportion_IC_PIR','proportion_D_TPC']

names2=['TotalPositiveCases', 'Deaths', 'Recovered','TestsPerformed','HospitalizedPatients', 'TotalHospitalizedPatients']

#for r in reg_list:
 #   df_pannel = regione3.loc[regione3.RegionName==r, :]
for n in names2:
    fig, axes = plt.subplots(1, 4, figsize=(11,5), sharex=True, sharey=True, dpi=100)
    for i, ax in enumerate(axes.flatten()[:4]):
        lag_plot(data[n], lag=i+1, ax=ax, c='firebrick')
        ax.set_title('Lag ' + str(i+1))
        


Le features selezionate mostrano che c'è correlazione, a livello nazionale, dato che non ho sparsità nelle occorrenze.

Seguono i grafici dei lag relativi alle features per ogni regione. 

In [ ]:
names = ['HospitalizedPatients', 'Deaths', 'IntensiveCarePatients', 'Recovered', 'HomeConfinement','TotalPositiveCases',
         'TestsPerformed']

for r in range(0, len(reg_list)):
    df_pannel = regione3.loc[regione3.RegionName==reg_list[r], :]
    print(reg_list[r])
    for n in names:
        fig, axes = plt.subplots(1, 4, figsize=(11,5), sharex=True, sharey=True, dpi=100)
        for i, ax in enumerate(axes.flatten()[:4]):
            lag_plot(df_pannel[n], lag=i+1, ax=ax, c='firebrick')
            ax.set_title('Lag ' + str(i+1))
        fig.suptitle('{}'.format(n), y=1.05)
        plt.show()

Per le features originali si evidenzia una correlazione tra le osservazioni con e senza lag. 

In [ ]:
names3 = ['ratio_IC_HP', 'ratio_TPC_TeP', 'ratio_TCP_Pop_gg', 'proportion_HC_TPC', 
          'proportion_CPC_TPC','proportion_R_TPC', 'proportion_IC_PIR','proportion_D_TPC']

for r in range(0, len(reg_list)):
    df_pannel = regione3.loc[regione3.RegionName==reg_list[r], :]
    print(reg_list[r])
    for n in names3:
        fig, axes = plt.subplots(1, 4, figsize=(11,5), sharex=True, sharey=True, dpi=100)
        for i, ax in enumerate(axes.flatten()[:4]):
            lag_plot(df_pannel[n], lag=i+1, ax=ax, c='firebrick')
            ax.set_title('Lag ' + str(i+1))
        fig.suptitle('{}'.format(n), y=1.05)
        plt.show()

Si evidenzia che è presente una maggior correlazione nelle osservazioni con lag per le features derivate relative a:  ratio_TCP_Pop_gg, proportion_IC_PIR, ratio_TPC_TeP

La forma del diagramma di lag fornisce indizi sulla struttura sottostante dei dati. Se è presente una forma lineare o si ha una forma tendente al lineare, questo suggerisce che probabilmente un modello di autoregressione è una scelta corretta.

Di seguito vengono riportati i grafici di autocorrelazione e autocorrelazione parziale.

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

names4 = ['HospitalizedPatients', 'IntensiveCarePatients', 'HomeConfinement','TotalPositiveCases', 'TestsPerformed']

for r in range(0, len(reg_list)):
    df_pannel = regione3.loc[regione3.RegionName==reg_list[r], :]
    print(reg_list[r])
    for n in names4:
        fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 100)
        plot_acf(df_pannel[n].tolist(), lags=20, ax=axes[0])
        plot_pacf(df_pannel[n].tolist(), lags=20, ax=axes[1])
        fig.suptitle('{}'.format(n), y=1.05)
        plt.show()


I grafici dei valori per l'ACF rientrano nell'intervallo di confidenza del 95% per lag> 2, rappresentato dalla banda azzurra, il che evidenzia la mancanza di autocorrelazione tra le osservazioni, di ogni feature analizzata, al di sopra di quel valore di lag. 
L'andamento di ogni feature sembra simile per ogni regione.

In [ ]:
names5 = ['ratio_IC_HP', 'ratio_TPC_TeP', 'ratio_TCP_Pop_gg', 'proportion_HC_TPC','proportion_CPC_TPC', 'proportion_IC_PIR']

for r in range(0, len(reg_list)):
    df_pannel = regione3.loc[regione3.RegionName==reg_list[r], :]
    print(reg_list[r])
    for n in names5:
        fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 100)
        plot_acf(df_pannel[n].tolist(), lags=20, ax=axes[0])
        plot_pacf(df_pannel[n].tolist(), lags=20, ax=axes[1])
        fig.suptitle('{}'.format(n), y=1.05)
        plt.show()

Analoghe considerazioni vengono effettuate per le features derivate analizzate.
Gli andamenti questa volta sono variabili per regione nel campo della stessa feature considerata.
Anche in questo caso per lag>2 si osserva una mancanza di autocorrelazione tra le osservazioni delle features.

Dall'osservazione del diagramma dell'autocorrelazione parziale si può intuire quanti lag passati includere nell'equazione di previsione di un modello auto-regressivo (AR). Nella maggior parte dei casi per lag>1 si rientra nella banda di confidenza, quindi il modello AR sarà di tipo 1.

---
## Time Series <a name="ts"></a>

In questa sezione ci dedichiamo alle seguenti analisi relative alle time series:
1. Visualizzazione trend 
2. Visualizzazione seasonality
3. Test stazionarietà
---

Utilizziamo il dataframe completo regione3, aggiungendo alcune features che specificano il giorno della settimana e il numero di settimana dell'anno. Per questo lasciamo invariato il dataframe originale e lavoriamo su un dataset temporaneo chiamato ts_df.

In [ ]:
ts_df=regione3.copy()

In [ ]:
ts_df.head()

Aggiungiamo le colonne day_of_week e week_of_year

In [ ]:
ts_df['day_of_week'] = ts_df.Date.apply(lambda x: x.dayofweek)
ts_df['week_of_year'] = ts_df.Date.apply(lambda x: x.weekofyear)

Stampiamo le prime cinque righe del dataframe appena modificato per vedere che le nuove colonne siano state effettivamente aggiunte.

In [ ]:
ts_df.head()

---
**Visualizzazione Trend**
---


Iniziamo lo studio delle time series analizzandone i grafici e vedendo se esistono trend.

Come primo passo definiamo due funzioni utili per disegnare i grafici relativi agli andamenti delle time series.

In [ ]:
# Draw Plot
def plot_df(df, x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
#Disegna i grafici di tutte le regioni in relazione ad una singola feature.
def plot_ts_regions(df, x_ax, y_ax, regionlist, frequency='dayly'):
    for r in range(0, len(regionlist)):
        temp1 = df.loc[df.RegionName==regionlist[r], :]
        plot_df(temp1, x=temp1[x_ax], y=temp1[y_ax], title=frequency+' trend of '+y_ax+' '+reg_list[r])

Iniziamo generando i grafici relativi alle principali features, per tutte le regioni. Daremo poi delle considerazioni finali sulle informazioni che possiamo dedurre dai grafici generati.

**New positive cases**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'NewPositiveCases', reg_list)

**Hospitalized Patients**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'HospitalizedPatients', reg_list)

**Deaths**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'Deaths', reg_list)

**Intensive Care patients**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'IntensiveCarePatients', reg_list)

**Recovered**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'Recovered', reg_list)

**Home confinement**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'HomeConfinement', reg_list)

**Total Positive cases**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'TotalPositiveCases', reg_list)

**Tests performed**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'TestsPerformed', reg_list)

**Ratio IC_HP**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'ratio_IC_HP', reg_list)

**Ration TPC_TeP**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'ratio_TPC_TeP', reg_list)

**Ratio TCP_Pop_gg**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'ratio_TCP_Pop_gg', reg_list)

**Proportion HC_TPC**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'proportion_HC_TPC', reg_list)

**Proportion CPC_TPC**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'proportion_CPC_TPC', reg_list)

**Proportion R_TPC**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'proportion_R_TPC', reg_list)

**Proportion IC_PIR**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'proportion_IC_PIR', reg_list)

**Proportion D_TPC**

In [ ]:
plot_ts_regions(ts_df, 'Date', 'proportion_D_TPC', reg_list)

---
**Considerazioni finali sui trend**



* NewPositiveCases: si nota un trend in crescendo nelle regioni più colpite, nelle regioni meno colpite invece si alternano picchi positivi e negativi, mantenendo comunque un trend crescente.

* HospitalizedPatients: in questo caso il trend risulta crescente in tutte le regioni, con curve continue nelle regioni più colpite.

* Deaths: in generale il trend è crescente nelle regioni più colpite, come la Lombardia, mentre in quelle meno colpite si hanno periodi spesso costanti, seguiti da alcuni picchi. Da notare che l'unica regione con valore costante a 0 è la Basilicata.

* IntensiveCarePatients: trend crescente per tutte le regioni.

* Recovered: trend crescente in tutte le regioni, con una maggiore ripidità nelle regioni meno colpite.

* HomeConfinement: trend crescente in tutte le regioni.

* TotalPositiveCases: trend crescente in tutte le regioni.

* TestsPerformed: trend crescente in tutte le regioni. 

* ratio_IC_HP: non è identificabile un vero e proprio trend, solo la regione Veneto sembra avere un trend in decrescita.

* ratio_TPC_TeP: crescente per tutte le regioni

* ratio_TCP_Pop_gg: crescente per tutte le regioni

* proportion_HC_TPC: in generale è crescente per quasi tutte le regioni, eccetto il Piemonte che ha un trend decrescente.

* proportion_CPC_TPC: Decrescente per le regioni con il maggior numero di casi, come Lombardia, Piemonte, Veneto ed Emilia Romagna, mentre nelle restanti abbiamo trend stabili, quasi costanti.

* proportion_R_TPC: abbiamo la Basilicata costante a 0, mentre per le altre regioni abbiamo un trend in generale decrescente per le regioni meno colpite e crescente invece per quelle più colpite.

* proportion_IC_PIR: in generale crescente in tutte le regioni

* proportion_D_TPC: in generale crescente in tutte le regioni, ad eccezione della Basilicata, costante a 0.

---
Visualizzazione Stagionalità
---

Come nella sezione relativa alla visualizzazione dei trend, anche qui definiamo delle funzioni utili alla creazione dei grafici.

In [ ]:
def weekly_seasonality_plot(df, featureName, reg_list, weeks, colors, week_dict):
    for r in range(0, len(reg_list)):
        temp1 = df.loc[df.RegionName==reg_list[r], :]
        print(reg_list[r])
        plt.figure(figsize=(14,10), dpi= 100)
        for i, y in enumerate(weeks):
            if i >= 0:        
                plt.plot('day_of_week', featureName, data=temp1.loc[temp1.week_of_year==y, :], color=colors[i], label=y)
                plt.text(temp1.loc[temp1.week_of_year==y, :].shape[0]-.9, temp1.loc[temp1.week_of_year==y, featureName][-1:].values[0], week_dict[y], fontsize=12, color=colors[i])
        plt.gca().set(title=featureName+' seasonality check', xlabel='day', ylabel=featureName)
        plt.show()

In [ ]:
weeks = ts_df['week_of_year'].unique()
# Prep Colors
np.random.seed(100)
mycolors = np.random.choice(list(mpl.colors.XKCD_COLORS.keys()), len(weeks), replace=False)
week_dict={9:'1st week',10:'2nd week',11:'3rd week',12:'4th week'}

Anche qui, come nella sezione precedente, generiamo prima tutti i grafici, e poi ne analizziamo i risultati in seguito nelle considerazioni finali sulla stagionalità.

**Legenda**


**Settimane**

* 1st week->prima settimana dall'inizio del contagio in Italia
* 2nd week->seconda settimana dall'inizio del contagio in Italia
* 3rd week->terza settimana dall'inizio del contagio in Italia
* 4th week->quarta settimana dall'inizio del contagio in Italia


**Giorni**

* 0: Lunedì
* 1: Martedì
* 2: Mercoledì
* 3: Giovedì
* 4: Venerdì
* 5: Sabato
* 6: Domenica

**New Positive cases**

In [ ]:
weekly_seasonality_plot(ts_df, 'NewPositiveCases', reg_list, weeks, mycolors, week_dict)

**Hospitalized Patients**

In [ ]:
weekly_seasonality_plot(ts_df, 'HospitalizedPatients', reg_list, weeks, mycolors, week_dict)

**Deaths**

In [ ]:
weekly_seasonality_plot(ts_df, 'Deaths', reg_list, weeks, mycolors, week_dict)

**Intensive care patients**

In [ ]:
weekly_seasonality_plot(ts_df, 'IntensiveCarePatients', reg_list, weeks, mycolors, week_dict)

**Recovered**

In [ ]:
weekly_seasonality_plot(ts_df, 'Recovered', reg_list, weeks, mycolors, week_dict)

**Home confinement**

In [ ]:
weekly_seasonality_plot(ts_df, 'HomeConfinement', reg_list, weeks, mycolors, week_dict)

**Total positive cases**

In [ ]:
weekly_seasonality_plot(ts_df, 'TotalPositiveCases', reg_list, weeks, mycolors, week_dict)

**Tests performed**

In [ ]:
weekly_seasonality_plot(ts_df, 'TestsPerformed', reg_list, weeks, mycolors, week_dict)

**Ratio IC_HP**

In [ ]:
weekly_seasonality_plot(ts_df, 'ratio_IC_HP', reg_list, weeks, mycolors, week_dict)

**ratio TPC_TeP**

In [ ]:
weekly_seasonality_plot(ts_df, 'ratio_TPC_TeP', reg_list, weeks, mycolors, week_dict)

**Ratio TCP_Pop_gg**

In [ ]:
weekly_seasonality_plot(ts_df, 'ratio_TCP_Pop_gg', reg_list, weeks, mycolors, week_dict)

**Proportion HC_TPC**

In [ ]:
weekly_seasonality_plot(ts_df, 'proportion_HC_TPC', reg_list, weeks, mycolors, week_dict)

**Proportion CPC_TPC**

In [ ]:
weekly_seasonality_plot(ts_df,'proportion_CPC_TPC', reg_list, weeks, mycolors, week_dict)

**Proportion R_TPC**

In [ ]:
weekly_seasonality_plot(ts_df, 'proportion_R_TPC', reg_list, weeks, mycolors, week_dict)

**Proportion IC_PIR**

In [ ]:
weekly_seasonality_plot(ts_df, 'proportion_IC_PIR', reg_list, weeks, mycolors, week_dict)

**Proportion D_TPC**

In [ ]:
weekly_seasonality_plot(ts_df, 'proportion_D_TPC', reg_list, weeks, mycolors, week_dict)

**Considerazioni finali sulla stagionalità**

Per quanto riguarda la stagionalità, in generale è difficile affermare che sia presente in qualcuna delle features considerate, soprattutto per la mancanza di uno storico di dati ed il breve periodo che ricoprono i dati disponibili. Per provare a vedere se esiste una stagionalità, abbiamo considerato le settimane, provando a vedere se determinati giorni, ad esempio il sabato e la domenica, avessero andamenti comuni e accentuati, in positivo o in negativo. Per features ordinarie, che hanno trend esclusivamente crescenti, come ad esempio IntensiveCarePatients, si potrebbe notare un minimo di stagionalità, mentre per le feature derivate no. Sarebbe stato indicativo e utile trovare stagionalità nel numero di nuovi casi positivi, ad esempio se si fosse trovata una marcata stagionalità nei giorni del week-end, si poteva supporre che, siccome nel weekend le persone tendono ad uscire e a creare assembramenti, si ha un maggiore numero di nuovi positivi. Questo però non accade, difatti anche se avessimo trovato stagionalità, i tempi di incubazione del virus sono diversi per ogni persona, quindi incrementi stagionali del numero di positivi in un dato giorno della settimana potrebbero essere semplici coincidenze. Concludendo, per i dati attualmente disponibili secondo noi non ci sono prove a sufficienza per affermare che vi sia stagionalità.

---
Verifica stazionarietà
---

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries):
    """
    Check Stationariety of time series.
    Please use np.array or pd.series as Input with your TS data only
    """
    #Convert numpy array to pandas serie
    if type(timeseries) is np.ndarray:
        df_timeseries = pd.Series(timeseries) 
        
    try:
        #Determing rolling statistics
        rolmean = df_timeseries.rolling(window=12).mean()
        rolstd = df_timeseries.rolling(window=12).std()

        #Plot rolling statistics:
        orig = plt.plot(timeseries, color='blue',label='Original')
        mean = plt.plot(rolmean, color='red', label='Rolling Mean')
        std = plt.plot(rolstd, color='black', label = 'Rolling Std')
        plt.legend(loc='best')
        plt.title('Rolling Mean & Standard Deviation')
        plt.show(block=False)

        #Perform Dickey-Fuller test:
        print('Results of Dickey-Fuller Test:')

        dftest = adfuller(timeseries, autolag='AIC')
        dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
        for key,value in dftest[4].items():
            dfoutput['Critical Value (%s)'%key] = value
        
        # print(dfoutput)
    
        return dftest, dfoutput
    except Exception as message:
        print(f"Impossible to calc the stationariery of your TS: {message}")
        return None, None

In [ ]:
temp_stationary=ts_df.copy()
temp_stationary=temp_stationary[temp_stationary.RegionName == 'Lombardia']

In [ ]:
temp_stationary['Date']=pd.to_datetime(temp_stationary['Date'])

In [ ]:
x = temp_stationary['Date'].values
y1 = temp_stationary['NewPositiveCases'].values

In [ ]:
dftest, dfoutput = test_stationarity(y1)
dfoutput

In [ ]:
x = temp_stationary['Date'].values
y1 = temp_stationary['IntensiveCarePatients'].values

In [ ]:
dftest, dfoutput = test_stationarity(y1)
dfoutput

---
Detrend
---

In [ ]:
#This is our TS
df_detrend = ts_df.copy()
df_detrend=df_detrend[df_detrend.RegionName=='Lombardia']
df_detrend.index = df_detrend.Date
df_detrend = df_detrend.drop('Date',axis=1)
plt.plot(df_detrend.index, df_detrend.IntensiveCarePatients)
plt.xticks(rotation=90)

In [ ]:
from scipy import signal
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
detrended = signal.detrend(df_detrend.IntensiveCarePatients.values)
plt.plot(df_detrend.index, detrended)
plt.xticks(rotation=90)
plt.title('IntensiveCarePatients by subtracting the least squares fit', fontsize=16)